In [16]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import time


In [19]:

def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in r.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = ret.set_index('證券代號')
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret

In [20]:
data = {}
n_days = 2
date = datetime.datetime.now()
fail_count = 0
allow_continuous_fail_count = 5
while len(data) < n_days:

    print('parsing', date)
    # 使用 crawPrice 爬資料
    try:
        # 抓資料
        data[date.date()] = crawl_price(date)
        print('success!')
        fail_count = 0
    except:
        # 假日爬不到
        print('fail! check the date is holiday')
        fail_count += 1
        if fail_count == allow_continuous_fail_count:
            raise
            break
    
    # 減一天
    date -= datetime.timedelta(days=1)
    time.sleep(10)

parsing 2020-01-09 15:10:54.589765
success!
parsing 2020-01-08 15:10:54.589765
success!


In [25]:
data.items()

dict_items([(datetime.date(2020, 1, 9),           證券名稱      成交股數   成交筆數       成交金額     開盤價     最高價     最低價     收盤價  \
證券代號                                                                         
1101        台泥  17899047  4,299  777095165   43.40   43.60   43.30   43.45   
1101B     台泥乙特      1000      1      53800   53.80   53.80   53.80   53.80   
1102        亞泥   5980170  2,859  291785676   48.55   49.00   48.50   48.95   
1103        嘉泥   1456880    655   31193118   21.40   21.55   21.25   21.55   
1104        環泥    397761    189    7579232   19.05   19.10   19.00   19.05   
1108        幸福    120076     55     981266    8.15    8.22    8.15    8.18   
1109        信大    455064    198    9282248   20.35   20.50   20.30   20.40   
1110        東泥    119917     24    2067121   17.30   17.30   17.20   17.30   
1201        味全   1060678    536   25801057   24.55   24.55   24.25   24.25   
1203        味王     33876     21     918359   27.15   27.15   27.05   27.10   
1210        大成   1931609

In [21]:
close = pd.DataFrame({k:d['收盤價'] for k,d in data.items()}).transpose()
close.index = pd.to_datetime(close.index)
close

,1101,1101B,1102,1103,1104,1108,1109,1110,1201,1203,...,9940,9941,9941A,9942,9943,9944,9945,9946,9955,9958
2020-01-08,43.40,53.80,48.55,21.30,19.05,8.15,20.30,17.30,24.25,27.15,...,31.45,114.50,54.10,69.60,70.20,17.65,44.10,27.00,22.00,68.00
2020-01-09,43.45,53.80,48.95,21.55,19.05,8.18,20.40,17.30,24.25,27.10,...,31.45,114.50,54.10,69.70,70.50,17.70,44.65,27.20,20.95,68.50


In [ ]:
df = data[datetime.date(2020,1,7)]
df